## B. To obtain specific yield values for quality-controlled groundwater levels

In [44]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import rioxarray
from sklearn.cluster import KMeans

Quality_controlled_well = pd.read_csv('/give_path_to/example_data/2_filtered_CGWB_all_wells.csv')
bhanja_aquifertype = rioxarray.open_rasterio("/give_path_to/example_data/2_hydrolgeology_map_georeferenced.tif")
lat_list = Quality_controlled_well.LAT
long_list = Quality_controlled_well.LON

i = 0
val_list = list()

for i in tqdm(range(len(lat_list))):
    lat = lat_list[i]
    long = long_list[i]
    
    try:
        val = bhanja_aquifertype.sel(y = lat, x = long, method = 'nearest').values
    except:
        val = np.nan
    
    val_list.append(val)

val_list_np = np.array(val_list)    

kmeans = cluster.KMeans(n_clusters=8, init='k-means++',random_state=42)

kmeans = kmeans.fit(val_list_np)
kmeans_label = kmeans.labels_

# Reclassify and compute mean specific yield
reclassify_kmeans_label = np.array(kmeans_label, dtype='f')

reclassify_kmeans_label[kmeans_label == 0] = 0.023 # Fractured Crystalline Gray 
reclassify_kmeans_label[kmeans_label == 1]= np.nan  # Black gray ignore 
reclassify_kmeans_label[kmeans_label == 2]= 0.13 #    Unconsolidated Sedimentary  Yellow
reclassify_kmeans_label[kmeans_label == 3] = 0.043 #Consolidated Sedimentary, permeable(blue)
reclassify_kmeans_label[kmeans_label == 4] = 0.02 #Joined crystalline(cyan)
reclassify_kmeans_label[kmeans_label == 5]=  0.018 # Folded metasediments pink  
reclassify_kmeans_label[kmeans_label == 6]= 0.018  # sedimentary aquitards (orange)
reclassify_kmeans_label[kmeans_label == 7] = np.nan #To be ignored black


100%|█████████████████████████████████████| 2265/2265 [00:01<00:00, 1929.24it/s]


In [45]:
Quality_controlled_well["reference_sy"] =  reclassify_kmeans_label #reference_sy
Quality_controlled_well

,STATE,DISTRICT,SITE_TYPE,WLCODE,LON,LAT,Jan 2004,May 2004,Aug 2004,Nov 2004,...,Jan 2014,May 2014,Aug 2014,Nov 2014,Jan 2015,May 2015,Aug 2015,Nov 2015,geometry,reference_sy
0,TG,Adilabad,Dug Well,W04508,79.666667,19.145833,6.59,7.40,5.97,6.77,...,4.37,6.67,5.67,5.37,5.37,4.55,4.55,5.67,POINT (79.66666667 19.14583333),0.023
1,TG,Adilabad,Dug Well,W04174,79.845833,19.091667,NaN,6.06,3.36,4.26,...,3.16,4.96,2.96,3.06,3.31,9.06,4.72,3.51,POINT (79.84583333 19.09166667),0.023
2,TG,Adilabad,Dug Well,W04524,78.511111,19.080556,6.13,7.99,5.17,5.97,...,3.37,5.52,3.62,3.86,3.97,6.37,4.62,5.77,POINT (78.51111111 19.08055556),0.043
3,TG,Adilabad,Dug Well,W04525,78.583333,19.019444,NaN,6.10,2.90,4.40,...,4.50,4.45,2.65,3.00,4.80,6.65,2.57,5.90,POINT (78.58333333 19.01944444),0.043
4,TG,Adilabad,Dug Well,W04516,79.047222,19.094444,5.36,6.95,5.90,6.60,...,2.70,2.30,2.95,1.30,3.27,5.27,4.80,1.25,POINT (79.04722222 19.09444444),0.043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,AP,Y.s.r.,Dug Well,W04224,78.766670,14.266670,5.33,7.00,NaN,3.85,...,6.52,5.97,6.60,6.55,4.36,8.05,8.71,7.27,POINT (78.76667 14.26667),0.023
2261,AP,Y.s.r.,Bore Well,W04654,78.598060,14.845560,26.00,26.70,25.38,20.35,...,NaN,21.72,20.40,10.95,6.33,28.90,26.05,23.90,POINT (78.59806 14.84556),0.018
2262,AP,Y.s.r.,Bore Well,W04002,78.700830,13.945560,6.11,6.25,5.85,5.50,...,9.67,NaN,23.95,9.53,3.98,8.25,10.81,5.45,POINT (78.70083 13.94556),0.023
2263,AP,Y.s.r.,Dug Well,W04647,78.966670,13.900000,11.31,11.38,11.90,10.03,...,10.59,9.23,8.83,10.42,10.77,10.18,11.89,NaN,POINT (78.96667 13.9),0.023
